In [1]:
# Imports
import numpy as np
import pandas as pd

import os
os.getcwd()

'/home/peisenha/ownCloud/office/workspace/software/repositories/organizations/restudToolbox/package/development/working'

## Simulation

I will now manually check the simulation of some samples to see if the problem is in the simulation.

In [2]:
df = pd.read_csv('data.respy.dat', delim_whitespace=True, header=0, na_values='.')
df.set_index(['Identifier', 'Period'], drop=False, inplace=True)


Index(['Identifier', 'Period', 'Choice', 'Wage', 'Experience_A',
       'Experience_B', 'Years_Schooling', 'Lagged_Activity', 'Type',
       'Total_Reward_1', 'Total_Reward_2', 'Total_Reward_3', 'Total_Reward_4',
       'Systematic_Reward_1', 'Systematic_Reward_2', 'Systematic_Reward_3',
       'Systematic_Reward_4', 'Shock_Reward_1', 'Shock_Reward_2',
       'Shock_Reward_3', 'Shock_Reward_4', 'Discount_Rate', 'General_Reward_1',
       'General_Reward_2', 'Common_Reward', 'Immediate_Reward_1',
       'Immediate_Reward_2', 'Immediate_Reward_3', 'Immediate_Reward_4'],
      dtype='object')

I want to calculate the final level of education by individual.

In [24]:

    return edu_final
    
    print(edu_start, edu_final)
    
    



11.300000000000001